In [1]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
# %matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

import pickle

2023-02-15 15:45:10.822404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 15:45:11.096345: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-15 15:45:11.149673: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-15 15:45:11.149700: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [3]:
lmap = compose(list, map)
ltake = compose(list, take)

In [4]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [5]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [6]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [7]:
ingredients = list(df['ingredients'])
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [8]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('соль', 20427),
 ('куриное яйцо', 11621),
 ('сахар', 10953),
 ('молотый черный перец', 10877),
 ('сливочное масло', 10323),
 ('пшеничная мука', 9950),
 ('чеснок', 9558),
 ('оливковое масло', 7932),
 ('репчатый лук', 7272),
 ('растительное масло', 6456)]

In [9]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

# it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
# it = lens.Each().Each().modify(lemmatize)(it)
# it[:10]

### Токенизация

In [10]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [11]:
stopwords = nltk.corpus.stopwords.words('russian')

In [12]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [13]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [14]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [15]:
#ОЧЕНЬ ДОЛГО
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

In [16]:
all_words_lemmatized_list = list(all_words_lemmatized)

#Добавил UNK и PAD
all_words_lemmatized_list = ['PAD', 'UNK']+all_words_lemmatized_list
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['PAD',
 'UNK',
 'añejo',
 'baby',
 'baileys',
 'bambino',
 'bean',
 'biogourmet',
 'biscoff',
 'blue']

In [17]:
#ДОЛГО + много оперативки
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [18]:
ft = fasttext.load_model('cc.ru.300.bin')
# fasttext.util.reduce_model(ft, 100)
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [19]:
word_embeddings_dict = {}
for word in all_words_lemmatized_list:
    if word  == 'PAD':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[1, 0]))
    elif word  == 'UNK':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[0, 1]))
    else:
        word_embeddings_dict[word] = np.concatenate((ft.get_word_vector(word), [0, 0]))

In [20]:
first(word_embeddings_dict.items())
# np.concatenate((ft.get_word_vector(word), [0, 0])).shape

('PAD',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

Расширяем разметрости для PAD и UNK

In [21]:
dim += 2

In [22]:
# Освободаем оперативку
del ft

### Векторы для ингредиентов

In [23]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
#Добавил UNK и PAD
all_ingredients_list = [('PAD',), ('UNK',)]+all_ingredients_list
all_ingredients_list 
ingredients_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingredients_numbering_dict.items())

[(('PAD',), 0),
 (('UNK',), 1),
 (('карри', 'мадрас'), 2),
 (('медовый', 'уксус'), 3),
 (('мускат', 'флёрдоранж'), 4),
 (('коричневый', 'рис', 'басмати'), 5)]

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [26]:
ingredients_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingredients_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingredients_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingredients_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(10, ingredients_embeddings_dict.items())[3]


(('медовый', 'уксус'),
 array([ 8.10408592e-02, -3.42131844e-02, -1.09642588e-01,  3.87744457e-02,
         2.64366776e-01, -1.52668823e-01,  6.37506302e-02, -5.08788973e-03,
        -1.00130826e-01, -5.13826795e-02, -8.41145720e-02,  1.72422249e-02,
        -1.71841584e-01, -4.65097576e-02,  1.09288547e-01,  1.03309392e-01,
        -1.58844218e-02, -6.43558726e-02, -3.01534429e-01, -1.68753728e-01,
         2.26348445e-01,  1.72121078e-02, -1.04646690e-01,  1.36713348e-01,
        -2.48688553e-02, -1.48994848e-02,  4.19593833e-02,  9.44069736e-02,
         1.02722544e-01,  9.99094248e-02,  1.52468826e-02,  9.76731628e-03,
         4.43267375e-02,  7.43295364e-02, -1.07716102e-01, -6.12843912e-02,
         8.70754458e-02, -1.54759124e-01, -5.38981296e-02,  4.12468687e-02,
         1.76861048e-01, -4.08888729e-02,  8.88290666e-02,  7.41030434e-02,
         6.54427807e-02,  6.24493230e-02, -7.47840917e-02,  5.55224791e-02,
         1.70922741e-01,  4.10169903e-02,  1.74585961e-01, -1.424

Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [27]:
ingredients_by_recipe_embedding = [
    sum(ingredients_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(ingredients_by_recipe_embedding)

41438

## Данные для нейросети

Всевозможные метки (course):

In [28]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

{'Выпечка и десерты': 0,
 'Завтраки': 1,
 'Закуски': 2,
 'Напитки': 3,
 'Основные блюда': 4,
 'Паста и пицца': 5,
 'Салаты': 6,
 'Соусы и маринады': 7,
 'Супы': 8}

Группируем кухни:

In [32]:
df_cuisines = pd.read_excel(data_path.joinpath('Сортировка_кухонь.xlsx'))
cuisines_groups = {}
for column in df_cuisines.columns:
    series = df_cuisines[column]
    cuisines_groups[str(column).strip()] = set(map(str.strip, list(series[series.notnull()])))
del cuisines_groups['Unnamed: 1']
inverse_cuisines_groups = {cuisine: cuisine_group
                           for (cuisine_group, cuisines) in cuisines_groups.items()
                           for cuisine in cuisines}
inverse_cuisines_groups

{'Скандинавская кухня': 'Скандинавская',
 'Киргизская кухня': 'Центральноазиатская',
 'Китайская кухня': 'Восточноазиатская',
 'Ирландская кухня': 'Европейская',
 'Бразильская кухня': 'Южноамериканская',
 'Польская кухня': 'Европейская',
 'Австрийская кухня': 'Европейская',
 'Алжирская кухня': 'Африканская',
 'Британская кухня': 'Европейская',
 'Югославская кухня': 'Балканская',
 'Немецкая кухня': 'Европейская',
 'Бурятская кухня': 'Центральноазиатская',
 'Абхазская кухня': 'Кавказская',
 'Португальская кухня': 'Европейская',
 'Камбоджийская кухня': 'Юго-Восточная Азия',
 'Кухня Вестероса': 'Европейская',
 'Сицилийская кухня': 'Европейская',
 'Восточно-индийская кухня': 'Юго-Восточная Азия',
 'Тайская кухня': 'Юго-Восточная Азия',
 'Литовская кухня': 'Европейская',
 'Египетская кухня': 'Африканская',
 'Еврейская кухня': 'Ближний Восток',
 'Албанская кухня': 'Балканская',
 'Афганская кухня': 'Центральноазиатская',
 'Норвежская кухня': 'Скандинавская',
 'Одесская кухня': 'Славянская',
 '

In [35]:
cuisines_set = set(inverse_cuisines_groups.values())
# print(cuisines_set)
all_cuisines = {name: num for num, name in enumerate(cuisines_set)}
all_cuisines

{'Неклассифицируемое': 0,
 'Европейская': 1,
 'Славянская': 2,
 'Североамериканская': 3,
 'Ближний Восток': 4,
 'Африканская': 5,
 'Кавказская': 6,
 'Южноамериканская': 7,
 'Балканская': 8,
 'Центральноазиатская': 9,
 'Восточноазиатская': 10,
 'Юго-Восточная Азия': 11,
 'Скандинавская': 12}

In [38]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'],
    'ingredient_embedding': ingredients_by_recipe_embedding,
    'course_mark': [all_courses[course] for course in df['course']],
    'cuisine_group_mark': [all_cuisines[inverse_cuisines_groups[cuisine]] for cuisine in df['cuisine']]
})
df_for_learning

,id,ingredient_embedding,course_mark,cuisine_group_mark
0,28195,"[0.611611146479845, -0.0743011748418212, -0.09...",0,7
1,28141,"[0.8909059994039126, -0.11706455610692501, -0....",0,7
2,27929,"[1.08976399153471, -0.4261779775843024, -0.389...",0,7
3,28192,"[1.1749160811305046, -0.5175759345293045, -0.3...",0,7
4,28463,"[0.6190221486613154, -0.14342674519866705, -0....",0,7
...,...,...,...,...
41433,43380,"[1.3148583127185702, -0.05140630202367902, -0....",8,2
41434,80446,"[1.0693077351897955, 0.24955729965586215, -0.1...",8,12
41435,136820,"[0.7934096870012581, 0.5892392117530107, -0.44...",8,12
41436,18014,"[0.834478304721415, 0.4054728071205318, 0.2355...",8,12


### Делаем tf-idf

In [63]:
recipe_series = pd.Series(index=df['id'], data=ingredients_lemmatized)
recipe_series

id
28195     [(банан,), (лимонный, сок), (молотый, корица),...
28141     [(сахар,), (кокосовый, молоко), (корень, имбир...
27929     [(клубника,), (манго,), (сахар,), (тёртый, цед...
28192     [(манго,), (тёмный, ром), (сок, лайм), (сахар,...
28463     [(клубника,), (сахарный, пудра), (сок, лайм), ...
                                ...                        
43380     [(белый, фасоль), (овощной, бульон), (свёкла,)...
80446     [(цветной, капуста), (морковь,), (картофель,),...
136820    [(горох,), (репчатый, лук), (гвоздик,), (свино...
18014     [(оливковый, масло), (репчатый, лук), (овощной...
49530     [(картофель,), (репчатый, лук), (мясной, фарш)...
Length: 41438, dtype: object

In [108]:
try:
    del tf
finally:
    tf = pd.DataFrame(index=df['id'].rename('recipe_id'), 
                      columns=all_ingredients_list,
                      data=0,
                      dtype=np.float64)
for index, count in tf.iterrows():
    for ingredient in recipe_series.loc[index]:
        count[ingredient] += 1
    count /= len(recipe_series.loc[index])
tf

,"(PAD,)","(UNK,)","(карри, мадрас)","(медовый, уксус)","(мускат, флёрдоранж)","(коричневый, рис, басмати)","(имбирный, сироп)","(свиной, жировой, сетка)","(филе, барабулька)","(белые, баклажан)",...,"(шоколадный, масло)","(гхи,)","(чай, мат)","(филе, скумбрия)","(свиной, черёво)","(глазурь, тёмный, шоколад, dr.oetker)","(черничный, шраб)","(сладкий, красный, лук)","(сухой, дрожжи, dr.oetker)","(мариновать, перец, чили)"
recipe_id,,,,,,,,,,,,,,,,,,,,,
28195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
idf = dict(zip(tf.columns, iterate(lambda _: 1, 1)))
for recipe in recipe_series.values:
    for ingredient in recipe:
        idf[ingredient] += 1
idf = pd.Series(index=pd.Index(idf.keys(), tupleize_cols=False), data=idf.values(), dtype=np.float64)
idf /= len(tf.index)
idf = -idf.apply(np.log)
idf

(PAD,)                                   10.631954
(UNK,)                                   10.631954
(карри, мадрас)                           9.533341
(медовый, уксус)                          9.938806
(мускат, флёрдоранж)                      9.533341
                                           ...    
(глазурь, тёмный, шоколад, dr.oetker)     9.022516
(черничный, шраб)                         9.938806
(сладкий, красный, лук)                   5.364095
(сухой, дрожжи, dr.oetker)                7.923903
(мариновать, перец, чили)                 7.992896
Length: 4406, dtype: float64

In [161]:
try:
    del tfidf
finally:
    tfidf = tf*idf
tfidf

,"(PAD,)","(UNK,)","(карри, мадрас)","(медовый, уксус)","(мускат, флёрдоранж)","(коричневый, рис, басмати)","(имбирный, сироп)","(свиной, жировой, сетка)","(филе, барабулька)","(белые, баклажан)",...,"(шоколадный, масло)","(гхи,)","(чай, мат)","(филе, скумбрия)","(свиной, черёво)","(глазурь, тёмный, шоколад, dr.oetker)","(черничный, шраб)","(сладкий, красный, лук)","(сухой, дрожжи, dr.oetker)","(мариновать, перец, чили)"
recipe_id,,,,,,,,,,,,,,,,,,,,,
28195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Все нужные данные переводим в pickle

In [162]:
dict_to_pickle = {
    'df_for_learning': df_for_learning,
    'all_courses': all_courses,
    'all_cuisines': all_cuisines,
    'word_embeddings_dict': word_embeddings_dict,
    'ingredients_numbering_dict': ingredients_numbering_dict,
    'dim': dim,
    'ingredients_lemmatized': ingredients_lemmatized,
    'all_ingredients_list': all_ingredients_list,
    'ingredients_embeddings_dict': ingredients_embeddings_dict,
    # 'tfidf': tfidf
}
with data_path.joinpath('processed_data_dict.pkl').open(mode='wb') as f:
    pickle.dump(dict_to_pickle, f)